# 04 Playbook Tables

Create playbook segments by discount tier, popularity, and cadence.

In [1]:
from pathlib import Path
import sys

def find_repo_root():
    cwd = Path().resolve()
    for candidate in (cwd,) + tuple(cwd.parents):
        if (candidate / 'src').is_dir() and (candidate / 'config.yaml').exists():
            return candidate
    return cwd

ROOT = find_repo_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import pandas as pd

from src.metrics import add_buckets, build_playbook_table

sales_path = ROOT / 'data/processed/sales.parquet'
sales = pd.read_parquet(sales_path)
print('sales cols:', len(sales.columns))
needed = [
    'sale_depth_max',
    'sale_duration_days',
    'baseline_players',
    'peak_lift_pct',
    'AUL',
    'decay_days_to_baseline',
    'discount_tier_bucket',
    'popularity_bucket',
    'cadence_bucket',
]
missing = [c for c in needed if c not in sales.columns]
print('Missing:', missing)

for c in ['peak_lift_pct', 'AUL', 'decay_days_to_baseline', 'baseline_players']:
    if c in sales.columns:
        print(c, 'null%:', float(sales[c].isna().mean()))

sales = add_buckets(sales)
playbook = build_playbook_table(sales)

out_raw = ROOT / 'reports/playbook_table_raw.csv'
out_raw.parent.mkdir(parents=True, exist_ok=True)
playbook.to_csv(out_raw, index=False)
print('Wrote', out_raw)

display(playbook.head(10))
print('Segments with NaN discount_tier_bucket (count):', playbook['discount_tier_bucket'].isna().sum())
display(playbook[playbook['discount_tier_bucket'].isna()].head())

pb = playbook.copy()
pb = pb[pb['discount_tier_bucket'].notna()]

MIN_N = 20
pb = pb[pb['n_sales'] >= MIN_N]

if pb.empty:
    MIN_N = 10
    pb = playbook.copy()
    pb = pb[pb['discount_tier_bucket'].notna()]
    pb = pb[pb['n_sales'] >= MIN_N]

pb['median_peak_lift_pct_capped'] = pb['median_peak_lift_pct'].clip(upper=1000)

stable = pb.sort_values('median_peak_lift_pct_capped', ascending=False)
out_stable = ROOT / 'reports/playbook_table_stable.csv'
stable.to_csv(out_stable, index=False)
print('Wrote', out_stable)
print(f'Stability gates: non-null discount_tier_bucket, n_sales >= {MIN_N}, lift capped at 1000 for ranking')

top = stable.head(10)
print(f'Top segments (filtered): n_sales >= {MIN_N}, non-null tier, lift capped for ranking')
display(top)

print('Key takeaways (filtered):')
if not top.empty:
    print('- Highest lift concentrated in', top.iloc[0]['discount_tier_bucket'], 'tier')
    print('- Strongest segments tend to be', top.iloc[0]['popularity_bucket'], 'popularity')
    print('- Cadence bucket mix:', top['cadence_bucket'].value_counts().to_dict())
else:
    print('- No stable segments met the thresholds; lower MIN_N or review bucketing.')


sales cols: 18
Missing: []
peak_lift_pct null%: 0.0007408504963698325
AUL null%: 0.0
decay_days_to_baseline null%: 0.35412653726478
baseline_players null%: 0.0
Wrote C:\Users\halol\Desktop\steam-discount-impact\reports\playbook_table_raw.csv


,discount_tier_bucket,popularity_bucket,cadence_bucket,n_sales,median_peak_lift_pct,iqr_peak_lift_pct,median_AUL,median_decay_days_to_baseline,median_lift_per_discount_point
0,0-10%,Q1,low,14,365.705195,13720.298094,19.909109,2.0,36.570520
1,0-10%,Q1,mid,6,82.187808,4122.825345,5.578535,2.0,8.218781
2,0-10%,Q1,high,3,166.666667,92.265676,1.826087,2.5,16.666667
3,0-10%,Q2,low,10,26.161235,26.858393,1.272953,1.0,2.616124
4,0-10%,Q2,mid,6,26.587925,6.631989,0.840831,1.0,2.658793
5,0-10%,Q2,high,7,28.741481,9.706819,0.681327,3.0,2.874148
6,0-10%,Q3,low,6,28.630588,24.104175,1.047360,2.0,2.863059
7,0-10%,Q3,mid,3,199.823755,185.021873,12.114712,2.0,19.982376
8,0-10%,Q3,high,3,27.742730,49.890349,0.460994,1.0,2.774273
9,0-10%,Q4,low,5,47.948696,245.848036,1.053281,5.0,4.794870


Segments with NaN discount_tier_bucket (count): 0


,discount_tier_bucket,popularity_bucket,cadence_bucket,n_sales,median_peak_lift_pct,iqr_peak_lift_pct,median_AUL,median_decay_days_to_baseline,median_lift_per_discount_point


Wrote C:\Users\halol\Desktop\steam-discount-impact\reports\playbook_table_stable.csv
Stability gates: non-null discount_tier_bucket, n_sales >= 20, lift capped at 1000 for ranking
Top segments (filtered): n_sales >= 20, non-null tier, lift capped for ranking


,discount_tier_bucket,popularity_bucket,cadence_bucket,n_sales,median_peak_lift_pct,iqr_peak_lift_pct,median_AUL,median_decay_days_to_baseline,median_lift_per_discount_point,median_peak_lift_pct_capped
37,51-75%,Q1,mid,650,239.071030,883.179599,9.604202,4.0,3.823199,239.071030
49,76-100%,Q1,mid,356,228.384000,757.829252,8.989899,4.0,2.723802,228.384000
38,51-75%,Q1,high,642,224.695122,865.220116,6.854052,3.0,3.674078,224.695122
48,76-100%,Q1,low,205,218.181818,709.934065,10.090909,4.0,2.518892,218.181818
36,51-75%,Q1,low,436,216.909231,584.013899,9.634100,4.0,3.396453,216.909231
50,76-100%,Q1,high,353,158.719616,443.935158,5.484888,3.0,1.932336,158.719616
24,26-50%,Q1,low,267,138.222961,328.484401,4.325669,4.0,3.171728,138.222961
25,26-50%,Q1,mid,198,135.606871,323.084419,6.801769,3.0,3.059920,135.606871
12,11-25%,Q1,low,31,128.467153,656.004460,6.248175,3.0,6.423358,128.467153
51,76-100%,Q2,low,226,124.155401,144.599188,5.242786,7.5,1.499322,124.155401


Key takeaways (filtered):
- Highest lift concentrated in 51-75% tier
- Strongest segments tend to be Q1 popularity
- Cadence bucket mix: {'low': 5, 'mid': 3, 'high': 2}


In [2]:
from datetime import datetime, timezone


def df_to_markdown(df):
    try:
        return df.to_markdown(index=False)
    except Exception:
        if df.empty:
            return "(no rows)"
        cols = [str(c) for c in df.columns]
        lines = []
        lines.append("| " + " | ".join(cols) + " |")
        lines.append("| " + " | ".join(["---"] * len(cols)) + " |")
        for row in df.itertuples(index=False, name=None):
            lines.append("| " + " | ".join("" if v is None else str(v) for v in row) + " |")
        return "\n".join(lines)


if 'playbook' not in globals() or 'top' not in globals() or 'MIN_N' not in globals():
    raise RuntimeError('Run the previous cell first to build playbook/top/MIN_N.')

nan_count = playbook['discount_tier_bucket'].isna().sum()
nan_sample = playbook[playbook['discount_tier_bucket'].isna()].head()
top_table = top.head(10)

lines = []
lines.append('# Notebook 04 Validation Snapshot')
lines.append('')
lines.append(f'Timestamp: {datetime.now(timezone.utc).isoformat()}')
lines.append('')
lines.append('## NaN Tier Count')
lines.append('')
lines.append(f'Segments with NaN discount_tier_bucket (count): {nan_count}')
lines.append('')
lines.append('## NaN Tier Sample')
lines.append('')
lines.append(df_to_markdown(nan_sample))
lines.append('')
lines.append('## Top Segments (Filtered)')
lines.append('')
lines.append(df_to_markdown(top_table))
lines.append('')
lines.append('## Filter Parameters')
lines.append('')
lines.append(f'- MIN_N: {MIN_N}')
lines.append('- Excluded NaN tiers: True (discount_tier_bucket.notna())')
lines.append('- Lift cap for ranking: 1000 (median_peak_lift_pct_capped)')

out_path = ROOT / 'reports/notebook04_validation_snapshot.md'
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text('\n'.join(lines), encoding='utf-8')
print('Wrote', out_path)


Wrote C:\Users\halol\Desktop\steam-discount-impact\reports\notebook04_validation_snapshot.md
